In [157]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [158]:
from sklearn.metrics import classification_report

https://medium.com/stanford-cs224w/graph-neural-networks-for-knowledge-tracing-ef31fdaa5f00

In [159]:
pd.set_option('display.max_columns', None)

In [160]:
df=pd.read_pickle('appmath.pkl')  

In [161]:
df2=pd.read_csv('prereq_edges.csv')

In [162]:
G=nx.Graph()

for _,i in df2.iterrows():    
       
        G.add_edge(i['source_lo_title'],i['dest_lo_title'])
        
subgraphs=[G.subgraph(i) for i in nx.connected_components(G)]

for num,sub in enumerate(subgraphs):
    
    nx.set_node_attributes(sub,num,'subGraphId')

nodeSubGraphId={}
for g in subgraphs:
    for node in g.nodes(data=True):
         nodeSubGraphId[node[0]]= node[1]['subGraphId']
            
subgraphsNodeCountDict={ num: g.number_of_nodes() for num,g in  enumerate(subgraphs)}

In [163]:
for col in df.columns:
    if df[col].dtype.name=='category':
        df[col]=df[col].astype(str)

In [164]:
df['loGraphID']=df.learning_objective_name.apply(lambda x: nodeSubGraphId.get(x) )

In [165]:
difficulty=df[~df.correct.isna()].groupby('atom_id')['correct'].mean().reset_index().rename(columns={'correct':'difficulty'})

In [166]:
difficulty

,atom_id,difficulty
0,000361fc-3126-4f25-93da-3ee3b66021c4,0.571429
1,000785e5-46df-4d0b-a12d-2283db4b8c7f,1.000000
2,000da431-9e35-4309-a3ae-cca9225b8d54,0.638554
3,00170926-4ba4-4d1e-bd86-0ac94107acb1,0.700000
4,001cfd36-68fa-4d74-b077-769f3bf2e0ad,0.712919
...,...,...
16441,fff0e6fc-e78f-41da-bcc8-88a6dceb6cc3,0.581481
16442,fff4801c-d828-453e-b1d4-64369f7b1048,0.666667
16443,fff652bc-b348-4054-9aab-8e67b98b022a,0.714286
16444,fffbe876-95cc-4dd0-9097-c657f446df6a,0.714286


In [167]:
df=df.merge(difficulty,how='left',on='atom_id')

In [168]:
loIndex={lo:i for i,lo in enumerate( df.learning_objective_name.unique())}
df['loIndex']=df['learning_objective_name'].apply(lambda x: loIndex[x])


df2['source_index']=df2.source_lo_title.map(loIndex)
df2['dest_index']=df2.dest_lo_title.map(loIndex)

#df2=df2[['source_index','dest_index']].reset_index(drop=True)
#loEdgeIndex=loEdgeMapping.values.transpose()

In [169]:
loEdgeMapping=df2[['source_index','dest_index']].reset_index(drop=True)
loEdgeMapping=loEdgeMapping.dropna(axis=0)
loEdgeIndex=loEdgeMapping.values.transpose()

In [170]:
loEdgeMapping.isna().sum()

source_index    0
dest_index      0
dtype: int64

In [171]:
atomIndex={atomId:num for num,atomId in enumerate( df.atom_id.unique())}

In [172]:
df['atomIndex']=df.atom_id.apply(lambda x : atomIndex[x] )

In [173]:
def atomClassifier(x):
    if x==None:
        return 'learningMaterial'
    else:
        return 'question'

In [174]:
df['atom_type']=df.correct.apply(lambda x:atomClassifier(x) )

In [175]:
df[df.atom_type=='question']

,graph_id,course_id,section_id,user_id,registration_id,goal_id,path_id,path_type,adaptive_behavior,topic_id,lo_to_concept_blacklist,interaction_end_time,concept_id,concept_partner_id,concept_name,retail_lo_id,learning_objective_name,atom_id,retail_atom_id,variation_id,sequence_created_at,parent_sequence_id,sequence_instance_id,sequence_ordinal,recommendation_id,source,state,type,correct,time_spent_answering_s,time_spent_on_instruction_s,goal_progress,work_remaining,status,target_status_and_progress,prev_concept_narrative,current_concept_narrative,focused_concept_id,prev_lo_narrative,current_lo_narrative,focused_lo_id,focused_topic_id,duration_s,is_target,loGraphID,difficulty,loIndex,atomIndex,atom_type
0,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,689c9d2c-b88e-40f1-9e5a-3a7193161cac,2dfdde50-72c3-4d49-8840-f0de82fedd0a,0012ff2b-871a-45d6-8e33-804f7464fa0a,0bbde3de-283b-4b08-a783-1a2942d07a2e,e359efc4-ed49-4227-a03e-c0c9358487d3,009f012a-30c9-42c5-94c6-3e05008ccfd0,REVIEW,None,d5743794-1f74-46ca-a538-8bf6c1cbb30b,NaN,2022-10-02 16:17:07.342,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,90bf68ef-fa40-4c57-9054-c417a848d897,5e457b03-88e7-4798-9c17-0c44a1c2d419,4ffcf121-73c3-4e21-adb9-d117faecffe9,2022-10-02 15:57:10.382,5baf1afa-5bc6-4c72-a7ac-b1bd4f180d22,c3b5bff2-e288-473d-bc1b-88c38be34d35,1,6.982368e+18,RECOMMENDATION,DONE,ASSESS,False,396.441010,24.492001,0.037663,-1.0,in_progress,{'53568360-e9fe-465f-a8fc-1c75027cba94': {'pro...,DIRECT_CHOOSE_TOPIC,DIRECT_DIAGNOSE_TOPIC,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,NONE,NONE,ff1bfcee-603f-4b08-924a-3852972fffab,tref-d5743794-1f74-46ca-a538-8bf6c1cbb30b,420.933014,True,3.0,0.525000,0,0,question
1,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,689c9d2c-b88e-40f1-9e5a-3a7193161cac,2dfdde50-72c3-4d49-8840-f0de82fedd0a,0012ff2b-871a-45d6-8e33-804f7464fa0a,0bbde3de-283b-4b08-a783-1a2942d07a2e,e359efc4-ed49-4227-a03e-c0c9358487d3,009f012a-30c9-42c5-94c6-3e05008ccfd0,REVIEW,None,d5743794-1f74-46ca-a538-8bf6c1cbb30b,NaN,2022-10-02 16:17:42.073,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,53568360-e9fe-465f-a8fc-1c75027cba94,Construct a truth table for a conditional stat...,49287c70-b63d-41be-9acc-24fd1f273158,614cad88-03e6-4c22-9689-aab68b3f23ba,5dd2e89c-bbcf-41ca-a91c-052010465d40,2022-10-02 16:17:07.893,a9af40ef-a988-4a30-b1fc-2344676d4cc7,408fc057-0815-4940-8ede-1ebe8207c1f0,1,6.982373e+18,RECOMMENDATION,DONE,ASSESS,True,26.042000,8.122000,0.533186,-1.0,in_progress,{'53568360-e9fe-465f-a8fc-1c75027cba94': {'pro...,DIRECT_DIAGNOSE_TOPIC,DIRECT_DIAGNOSE_TOPIC,b7d9982e-26f9-47c0-b76f-cc9dc1f0bfbe,NONE,NONE,ff1bfcee-603f-4b08-924a-3852972fffab,tref-d5743794-1f74-46ca-a538-8bf6c1cbb30b,34.164001,True,3.0,0.547619,0,1,question
3,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,689c9d2c-b88e-40f1-9e5a-3a7193161cac,2dfdde50-72c3-4d49-8840-f0de82fedd0a,0012ff2b-871a-45d6-8e33-804f7464fa0a,630b3513-7534-4ebc-8fe6-4625ad5ae796,4debeb6d-fc33-46ad-a19b-307c7ae270d0,0e2c0048-8974-4e02-93ce-443075bb7087,ADAPTIVE,PRACTICE,00b67632-3af0-47f7-ac19-77c8c686244b,NaN,2022-12-16 18:34:22.470,d02dcdf8-70dd-46db-bd8b-0894f303871d,e5dfa5f5-7ded-4b39-911e-b2b1737007a1,Determine the sample space of an experiment,e5dfa5f5-7ded-4b39-911e-b2b1737007a1,Determine the sample space of an experiment,1beef294-92f4-40f5-a340-b02f4cc9970c,a72b2188-29fd-4a11-9970-a0583361ccbf,6991ec82-96a0-4f31-9dd7-a2b8ab59c60f,2022-11-07 01:26:59.387,85e666eb-f520-4557-8fec-dfce666e74d3,c705f1e8-76da-4a5d-83fb-a8016058f580,1,6.995195e+18,RECOMMENDATION,DONE,ASSESS,False,87.829002,3.608000,0.007354,-1.0,in_progress,{'e5dfa5f5-7ded-4b39-911e-b2b1737007a1': {'pro...,DIRECT_CHOOSE_TOPIC,DIRECT_DIAGNOSE_TOPIC,d02dcdf8-70dd-46db-bd8b-0894f303871d,NONE,NONE,c85a591f-8169-4ea8-af31-f89a007f6a7e,tref-00b67632-3af0-47f7-ac19-77c8c686244b,9

In [176]:
df['correctBinary']=df.correct*1
df.atom_id=df.atom_id.astype(str)


In [177]:
def user_dataset(df,user):
    
    '''
    edges
    -------
    lo - atom
    atom - atom
    lo - learning materials (lm)
    atom - lm
    lm - atom
    lm - lm
    
    node / features
    ---------------
    
    lo - radnom number
    atom - dificulty , number of attempt
    lm - time spend 
        
    '''
  
    
    userDf=df[df.user_id==user]
    userDf=userDf.sort_values(by='interaction_end_time')
    #lo-atom
    loAtomEdgeIndex=userDf[userDf.atom_type=='question'][['loIndex','atomIndex']].values.transpose()
    #lo-lm
    loLmEdgeIndex=userDf[userDf.atom_type=='learningMaterial'][['loIndex','atomIndex']].values.transpose()
    
    userDf['attempt_count']=userDf.groupby(by=['atom_id']).cumcount()+1
    
    userDf['atom_type_shift']=userDf.atom_type.shift(-1)
    userDf['atomIndex_shift']=userDf.atomIndex.shift(-1)
    
    userDf= userDf[~userDf.atomIndex_shift.isna()]
    
    #atom-atom
    atomEdgeIndex=dftest[(dftest.atom_type=='question') & (dftest.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    #atom-lm
    atomLmEdgeIndex=dftest[(dftest.atom_type=='question') & (dftest.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    #lm-atom
    LmAtomEdgeIndex=dftest[(dftest.atom_type=='learningMaterial') & (dftest.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    #lm-lm
    LmLmEdgeIndex=dftest[(dftest.atom_type=='learningMaterial') & (dftest.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    
    #atom features
    atomFeatures = 
    
    
    
    
    

In [178]:

def user_dataset(df,user):  
    userDf=df[df.user_id==user]

    userDf=userDf.sort_values(by='interaction_end_time')
    userDf['attempt_count']=userDf.groupby(by=['atom_id']).cumcount()+1
    
    #atomIndex={atom:i for i,atom in enumerate( userDf[~userDf.correct.isna()].atom_id)}
    
    #learningMaterialIndex={atom:i for i,atom in enumerate( userDf[userDf.correct.isna()].atom_id)}

    #userDf['atomIndex']=np.arange(userDf.shape[0])

    atomsPerLO=userDf.groupby('loIndex')['atom_id'].count().to_dict()
    userDf['atomsPerLO']=userDf['loIndex'].apply(lambda x: atomsPerLO[x])

    edgeMap=userDf[['loIndex','atomIndex']]

    y=userDf.correctBinary.astype(int).to_numpy()

    loAtomEdgeIndex=edgeMap.values.transpose()

    atomAtom=edgeMap.reset_index(drop=True)[['atomIndex']]

    atomAtom['nextAtom']=atomAtom.atomIndex.shift(-1)

    atomAtom=atomAtom.dropna()

    atomEdgeIndex=atomAtom.values.transpose()

    atomFeatures=userDf.difficulty.to_numpy().reshape(-1,1)

    loFeatures=np.array(list(atomsPerLO.values())).reshape(-1,1)
    return  atomFeatures,loFeatures,loAtomEdgeIndex,atomEdgeIndex,y

In [179]:
dftest=df[(df.loGraphID==3) & (df.user_id=='0012ff2b-871a-45d6-8e33-804f7464fa0a' )].reset_index(drop=True)

In [180]:
dftest.atom_type.value_counts() 

question            293
learningMaterial     54
Name: atom_type, dtype: int64

In [181]:
#dftest['shift']=dftest.atomIndex.shift()

In [182]:
#dftest[['atom_type','shift','atomIndex']]

In [132]:
dftest[dftest.atom_type=='learningMaterial'].index

Int64Index([  2,  21,  30,  37,  38,  39,  40,  41,  46,  53,  63,  70,  80,
             86,  88, 100, 110, 122, 132, 149, 158, 162, 163, 171, 173, 179,
            181, 188, 190, 196, 198, 205, 224, 231, 242, 244, 251, 257, 264,
            270, 277, 284, 291, 293, 295, 303, 308, 310, 312, 314, 325, 332,
            339, 346],
           dtype='int64')

In [147]:
dftest['atom_type_shift']=dftest.atom_type.shift(-1)
dftest['atomIndex_shift']=dftest.atomIndex.shift(-1)
#dftest=dftest.dropna()

C:\Users\mpathirana\AppData\Local\Temp\ipykernel_6276\4120952831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftest['atom_type_shift']=dftest.atom_type.shift(-1)
C:\Users\mpathirana\AppData\Local\Temp\ipykernel_6276\4120952831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftest['atomIndex_shift']=dftest.atomIndex.shift(-1)


In [150]:
dftest[~dftest.atom_type_shift.isna()]

,atom_type,atomIndex,atom_type_shift,atomIndex_shift
0,question,0,question,1.0
1,question,1,learningMaterial,2.0
2,learningMaterial,2,question,31.0
3,question,31,question,32.0
4,question,32,question,33.0
...,...,...,...,...
340,question,787,question,788.0
341,question,788,question,789.0
342,question,789,question,790.0
343,question,790,question,791.0


In [145]:
dftest[(dftest.atom_type=='question') & (dftest.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    

array([[  0.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,
         41.,  42.,  43., 139., 140., 141., 142., 145., 146., 147., 148.,
        149., 150., 151., 154., 155., 156., 157., 158., 154., 155., 160.,
        166., 167., 168., 169., 170., 173., 174., 175., 176., 177., 178.,
        179., 180., 183., 184., 185., 186., 187., 190., 191., 192., 193.,
        194., 195., 196., 197., 200., 201., 202., 203., 208., 209., 210.,
        211., 212., 213., 214., 215., 216., 217., 220., 221., 222., 223.,
        193., 203., 194., 224., 227., 228., 229., 204., 230., 231., 232.,
        233., 234., 235., 238., 239., 240., 241., 242., 243., 244., 245.,
        248., 249., 250., 251., 252., 253., 254., 255., 256., 231., 257.,
        258., 259., 260., 261., 264., 265., 266., 267., 268., 269., 270.,
        273., 274., 635., 636., 637., 638., 639., 640., 645., 646., 647.,
        648., 653., 654., 655., 656., 657., 653., 635., 637., 648., 653.,
        659., 660., 661., 662., 665., 

In [146]:
from torch_geometric.data import HeteroData
import torch 


In [56]:
import torch_geometric.transforms as T

In [ ]:
#atomFeatures,loFeatures,loAtomEdgeIndex,atomEdgeIndex,y=user_dataset(user)

In [ ]:
def generate_data(atomFeatures,loFeatures,loAtomEdgeIndex,loEdgeIndex,atomEdgeIndex,y):  
    data = HeteroData()
    data['atom'].x=torch.tensor(atomFeatures).float()
    data['lo'].x= torch.tensor(loFeatures).float()
    data['lo','lo_atom','atom'].edge_index=torch.from_numpy(loAtomEdgeIndex).long()
    data['lo','lo_lo','lo'].edge_index= torch.from_numpy(loEdgeIndex).long()
    data['atom','atom_atom','atom'].edge_index=torch.from_numpy(atomEdgeIndex).long()

    data['atom'].y=torch.from_numpy(y).long()


    #split=T.RandomNodeSplit(num_test =5)
    #data=split(data)

    size=y.shape[0]
    num_test_nodes =int(y.shape[0]*.35)
    some_indices = torch.randperm(size)[:num_test_nodes]


    test_mask = torch.zeros(y.shape[0], dtype=torch.bool)
    test_mask[some_indices] = True
    data['atom'].test_mask = test_mask

    train_mask = torch.ones(y.shape[0], dtype=torch.bool)
    train_mask[some_indices] = False
    data['atom'].train_mask = train_mask

    normalize=T.NormalizeFeatures()
    data=normalize(data)
    return data